## Imports

In [ ]:
# Data analysis packages:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.3f'%x)
# pd.set_option('float_format', '{:f}'.format)
#from datetime import datetime as dt

# Visualization packages:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline

from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

import warnings
import itertools
import datetime as dt
from IPython.display import HTML # to see everything
plt.style.use('seaborn-darkgrid')
warnings.filterwarnings("ignore")

Import sklearn and statsmodels 

In [ ]:
# import sklearn 
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error

# import Statsmodels 
# from statsmodels.tsa.api import VAR
# from statsmodels.tsa.stattools import adfuller
# from statsmodels.tools.eval_measures import rmse, aic
# from statsmodels.tsa.seasonal import seasonal_decompose
# from statsmodels.graphics.gofplots import qqplot
# import statsmodels.api as sm

In [ ]:
# !pip install -user statsmodels

Import from Keras

In [ ]:
# !pip install tf-nightly-2.0-preview
import tensorflow as tf

# import Sequential
from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.layers import Input, Dense, LSTM, Embedding
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import (EarlyStopping, ModelCheckpoint,
                                               TensorBoard, ReduceLROnPlateau, CSVLogger)
# print 
print(tf.__version__)
print(tf.keras.__version__)

Import my functions

In [ ]:
# import plots as p
import supervised_learning as sl
# helper_functions as hf 
# import data_testing as dt
# import data_prep as dp
%load_ext autoreload
%autoreload 2

# class example

In [ ]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

def trend(time, slope=0):
    return slope * time

def seasonal_pattern(season_time):
    """Just an arbitrary pattern, you can change it if you wish"""
    return np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))

def seasonality(time, period, amplitude=1, phase=0):
    """Repeats the same pattern at each period"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)

def noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

time = np.arange(4 * 365 + 1, dtype="float32")
baseline = 10
series = trend(time, 0.1)  
baseline = 10
amplitude = 40
slope = 0.05
noise_level = 5

# Create the series
series = baseline + trend(time, slope) + seasonality(time, period=365, amplitude=amplitude)
# Update with noise
series += noise(time, noise_level, seed=42)

split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

window_size = 20
batch_size = 32
shuffle_buffer_size = 1000

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [ ]:
# print(type(x_train))
# print(x_train.shape)
# # print(type(x_valid))
# # print(x_valid.shape)

In [ ]:
windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

# Load Data -- Electricity

In [ ]:
office = pd.read_csv('data_folder/office_data.csv')#, index_col=['timestamp'], parse_dates=['timestamp'])
office = pd.read_csv('data_folder/office_1249.csv', index_col=['timestamp'], parse_dates=['timestamp'])
usecols =['meter', 'meter_reading', 'air_temperature', 'cloud_coverage', 'dew_temperature','precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
office = office[usecols]
office.head()

In [ ]:
# office[office.building_id ==1249].shape
office.shape

In [ ]:
office_temp = office.ffill()
office_data = office_temp.bfill()
office_data.head(3)

In [ ]:
# electricity
elec_consumption = office_data[office_data['meter']==0]
elec_consumption.drop(columns=['meter'], inplace=True)
print(elec_consumption.shape)
elec_consumption.head()

In [ ]:
# Check if dataframe has a zero element
# elec_consumption['wind_direction'].isin([0]).any().any()

### Box plots and outlier detection

In [ ]:
large = 22
med = 16
small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': large, # med
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'xtick.labelsize': large, #med,
          'ytick.labelsize': large, #med,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style('darkgrid')

In [ ]:
import seaborn as sns
sns.set(style="darkgrid")
# tips = sns.load_dataset("tips")
elec_consumption['meter_reading'].replace(to_replace=0, method='ffill', inplace=True)
ax = sns.boxplot(x=elec_consumption["meter_reading"])

In [ ]:
elec_consumption['meter_reading'].plot()

## Features and labels

In [ ]:
target_names = ['meter_reading', 'air_temperature']
# target_names = list(df_elc.columns) #['meter_reading', 'air_temperature'] # list(df_elc.columns)
shift_days = 1
shift_steps = shift_days * 24 # number of hours.
df_targets = elec_consumption[target_names].shift(-shift_steps)
df_targets.head(3)

In [ ]:
# check the raws from 24- 27
elec_consumption[target_names].iloc[24:27]

In [ ]:
# x_data
x_data = elec_consumption.values[0:-shift_steps]
print(type(x_data))
print('    Feature Shape:', x_data.shape)
 
# y_data
y_data = df_targets.values[:-shift_steps]
print(type(y_data))
print('    Target Shape:', y_data.shape)

In [ ]:
# These are the input-signals for the training- and test-sets:
num_data = len(x_data)
train_split = 0.9
num_train = int(train_split * num_data)
num_test = num_data - num_train
print('Training observations:', num_train)
print('Validation observations:', num_test)
x_train = x_data[0:num_train]
x_test  = x_data[num_train:]
print('Total observations:', (len(x_train) + len(x_test)))

In [ ]:
# Check the shape
x_train.shape, x_test.shape

In [ ]:
# These are the output-signals for the training- and test-sets:
y_train = y_data[0:num_train]
y_test = y_data[num_train:]
print('Total test observations:', (len(y_train) + len(y_test)))
y_train.shape, y_test.shape

In [ ]:
num_x_signals = x_data.shape[1] # 8
num_y_signals = y_data.shape[1] # 2
print('Number of input-signals:', num_x_signals, 'and shape:', x_train.shape )
print('Number of output-signals:', num_y_signals, 'and shape', y_train.shape)

## Scaled Data

In [ ]:
x_data[24:27] # I shifted the data by one day

In [ ]:
print('Min and Max x_train data')
print('    Min:', np.min(x_train))
print('    Max:', np.max(x_train))
x_scaler = MinMaxScaler()
x_scaled_train = x_scaler.fit_transform(x_train)
print('Min and Max x_train_scaled data')
print('    Min:', np.min(x_scaled_train))
print('    Max:', np.max(x_scaled_train))

In [ ]:
x_scaled_test = x_scaler.transform(x_test)
print('Min and Max x_test data')
print('    Min:', np.min(x_test))
print('    Max:', np.max(x_test))

# target MinMaxScaler
y_scaler = MinMaxScaler()
y_scaled_train = y_scaler.fit_transform(y_train)
y_scaled_test = y_scaler.transform(y_test)
print('Min and Max y_test_scaled data')
print('    Min:', np.min(y_scaled_test))
print('    Max:', np.max(y_scaled_test))

In [ ]:
print(x_scaled_train.shape)
print(y_scaled_train.shape)
print('-------')
print(x_scaled_test.shape)
print(y_scaled_test.shape)
# y_train_scaled

The data we have now is one long time series with 20 input signals so that each time step has 20 input signals and 3 output signals 

## Data Generator

In [ ]:
print(x_scaled_train.shape)
print(y_scaled_train.shape)

In [ ]:
def batch_generator(batch_size, sequence_length):
    """
    Generator function for creating random batches of training-data.
    """

    # Infinite loop.
    while True:
        # Allocate a new array for the batch of input-signals.
        x_shape = (batch_size, sequence_length, num_x_signals)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)

        # Allocate a new array for the batch of output-signals.
        y_shape = (batch_size, sequence_length, num_y_signals)
        y_batch = np.zeros(shape=y_shape, dtype=np.float16)

        # Fill the batch with random sequences of data.
        for i in range(batch_size):
            # Get a random start-index.
            # This points somewhere into the training-data.
            idx = np.random.randint(num_train - sequence_length)
            
            # Copy the sequences of data starting at this index.
            x_batch[i] = x_scaled_train[idx:idx+sequence_length]
            y_batch[i] = y_scaled_train[idx:idx+sequence_length]
        
        yield (x_batch, y_batch)

The above funcion will break the long time series signal into small sub sequences and create a batch so that we can use that for training the network 

```python
reshape input to be [samples, time steps, features]
```

In [ ]:
# reshape input to be [samples, time steps, features]
batch_size = 30 #72
sequence_length = shift_steps # 24 * 7

# create the batch-generator
generator = batch_generator(batch_size, sequence_length)

# test the batch-generator to see if it works 
x_batch, y_batch = next(generator)
print(x_batch.shape)
print(y_batch.shape)

we get a batch output where we have a batch size of xxx sequences, each sequence has yyy, and zzz input signals and zzz' output signals. 

In [ ]:
signal_list = list(elec_consumption.columns)
print('length of signals:', len(signal_list))

# signal plot for electricity consumption
batch = 0   # First sequence in the batch.
signal = 0  # First signal from the 8 input-signals.
seq = x_batch[batch, :, signal]
plt.plot(seq);

In [ ]:
# y plot for electricity consumption
seq = y_batch[batch, :, signal] # only two output signals
plt.plot(seq);

In [ ]:
x_scaled_test.shape

In [ ]:
validation_data = (np.expand_dims(x_scaled_test, axis=0),
                   np.expand_dims(y_scaled_test, axis=0))

In [ ]:
print('x_test_validation', validation_data[0].shape)
print('y_test_validation', validation_data[1].shape)

### Create the Recurrent Neural Network Architecture

In [ ]:
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

#### Initalize the RNN
Now let us inistantiate an RNN model

In [ ]:
# clear a model
tf.keras.backend.clear_session()

In [ ]:
# Initializing RNN 
model = Sequential()

In [ ]:
# model.summary()

#### Adding the LSTM layers and some Dropout regularization

In [ ]:
# Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units=50, return_sequences=True,
              input_shape=(None, num_x_signals,)))
# model.summary()

In [ ]:
model.add(Dense(num_y_signals, activation='sigmoid'))

if False:
    from tensorflow.python.keras.initializers import RandomUniform

    # Maybe use lower init-ranges.
    init = RandomUniform(minval=-0.05, maxval=0.05) # I ca

    model.add(Dense(num_y_signals,
                    activation='linear',
                    kernel_initializer=init))


### Loss Function

In [ ]:
warmup_steps = 50

load the loss_mse_warmup function below

In [ ]:
# %load -r 41-63 supervised_learning.py

### Compile Model

In [ ]:
optimizer = RMSprop(lr=1e-3)
# optimizer = tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9)

In [ ]:
# model.compile(loss=loss_mse_warmup, optimizer=optimizer, metrics=['acc'])
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['acc'])
model.summary()

### Callback Functions

In [ ]:
# path_checkpoint = '23_checkpoint.keras'
path_checkpoint = 'weights/model_weights.best.{epoch:02d}-{val_loss:.2f}.hdf5'
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                     monitor='val_loss',
                                      verbose=1,
                                      save_weights_only=True,
                                      save_best_only=True)
callback_early_stopping = EarlyStopping(monitor='val_loss',
                                        patience=5, verbose=1)

In [ ]:
callback_tensorboard = TensorBoard(log_dir='./logs/',
                                   histogram_freq=0,
                                   write_graph=False) #'./23_logs/'

In [ ]:
callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1,
                                       min_lr=1e-4,
                                       patience=0,
                                       verbose=1)

In [ ]:
csv_logger = CSVLogger('logs/training_model.csv')

In [ ]:
# callback lists 
callbacks = [callback_early_stopping,
             callback_checkpoint,
             callback_tensorboard,
             callback_reduce_lr,
             csv_logger]

### Fit the RNN to the training set 

In [ ]:
%%time
model.fit_generator(generator=generator,
                    epochs=20,
                    steps_per_epoch=100,
                    validation_data=validation_data,
                    callbacks=callbacks)

In [ ]:
# save the model in h5 format
# model.save('weights/first_model.h5')

In [ ]:
# load the model
lastew = load_model('weights/first_model.h5')

In [ ]:
# evaluate training set
x_scaled_train.shape, y_scaled_train.shape
x_sis = np.expand_dims(x_scaled_train, axis=0)
y_sis = np.expand_dims(y_scaled_train, axis=0)
lastew.evaluate(x_sis, y_sis)

In [ ]:
# evaluate test set
lastew.evaluate(validation_data[0], validation_data[1])
print('validation set should be a 3D arrray', 
      validation_data[0].shape, validation_data[1].shape)

In [ ]:
print(x_scaled_train.shape)
print(y_scaled_train.shape)
print('-----')
print(x_scaled_test.shape)
print(y_scaled_test.shape)
type(x_scaled_train)

In [ ]:
result = model.evaluate(x=np.expand_dims(x_scaled_test, axis=0),
                        y=np.expand_dims(y_scaled_test, axis=0))

In [ ]:
print("loss (test-set):", result)

In [ ]:
# If you have several metrics you can use this instead.
if False:
    for res, metric in zip(result, model.metrics_names):
        print("{0}: {1:.3e}".format(metric, res))

load plot comparision plot

In [ ]:
# %load -r 30-66 visualization.py
def plot_comparison(start_idx, length=100, train=True):
    
    if train:
        # Use training-data.
        x = x_scaled_train
        y_true = y_train
    else:
        # Use test-data.
        x = x_scaled_train
        y_true = y_test
    
    end_idx = start_idx + length
    
    x = x[start_idx:end_idx]
    y_true = y_true[start_idx:end_idx]
    
    x = np.expand_dims(x, axis=0)

    y_pred = model.predict(x)
    
    y_pred_rescaled = y_scaler.inverse_transform(y_pred[0])
    
    for signal in range(len(target_names)):
        signal_pred = y_pred_rescaled[:, signal]

        signal_true = y_true[:, signal]

        plt.figure(figsize=(15,5))
        
        plt.plot(signal_true, label='true')
        plt.plot(signal_pred, label='pred')
       
        p = plt.axvspan(0, warmup_steps, facecolor='black', alpha=0.15)
       
        plt.ylabel(target_names[signal])
        plt.legend()
        plt.show()

In [ ]:
plot_comparison(start_idx=4000, length=1000, train=True)

### Example from test set

In [ ]:
plot_comparison(start_idx=0, length=1000, train=False)

# # 2nd_LSTM Model
this model uses an upgrade on batch size
- softmax activation
- optimizer adam 
- batch size 72

In [ ]:
# reshape input to be [samples, time steps, features]
batch_size_2 = 30
# sequence_length = shift_steps # 24 * 7

# create the batch-generator
generator_2 = batch_generator(batch_size_2, sequence_length)

# test the batch-generator to see if it works 
x_batch_2, y_batch_2 = next(generator_2)
print(x_batch_2.shape)
print(y_batch_2.shape)

In [ ]:
# reshape input to be [samples, time steps, features]
# batch_size_2 = 30
# sequence_length = shift_steps # 24 * 7

# create the batch-generator
# generator_2 = batch_generator(batch_size_2, sequence_length)

# test the batch-generator to see if it works 
x_batch, y_batch = next(generator)
print(x_batch.shape)
print(y_batch.shape)

In [ ]:
# clear a model
tf.keras.backend.clear_session()

In [ ]:
# Initializing RNN 
model2 = Sequential()

#### Adding the LSTM layers and some Dropout regularization

In [ ]:
# Adding the first LSTM layer and some Dropout regularisation
model2.add(LSTM(units=50, return_sequences=True,
              input_shape=(None, num_x_signals,)))
# model.summary()

In [ ]:
# Adding the second LSTM layer and some Dropout regularisation
model2.add(LSTM(units=50, return_sequences=True))
# model.summary()

In [ ]:
model2.add(Dense(num_y_signals, activation='sigmoid'))

if False:
    from tensorflow.python.keras.initializers import RandomUniform

    # Maybe use lower init-ranges.
    init = RandomUniform(minval=-0.05, maxval=0.05) # I ca

    model2.add(Dense(num_y_signals,
                    activation='linear',
                    kernel_initializer=init))

In [ ]:
optimizer = RMSprop(lr=1e-3)
# optimizer = tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9)

In [ ]:
# model.compile(loss=loss_mse_warmup, optimizer=optimizer, metrics=['acc'])
model2.compile(loss='mean_squared_error', optimizer='adam', metrics=['acc'])
model2.summary()

### Fit the RNN to the training set 

In [ ]:
%%time
model.fit_generator(generator=generator,
                    epochs=20,
                    steps_per_epoch=100,
                    validation_data=validation_data,
                    callbacks=callbacks)

In [ ]:
generator_2[0]

In [ ]:
# !pip install fbprophet --user

In [ ]:
# from fbprophet import Prophet
# import logging 
# logging.getLogger().setLevel(Logging.ERRORProPr)
# prophet = Prophet()

In [ ]:
# !pip install progressbar2 --user

#### Finding Fedaral Holidays

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

In [ ]:
df = df_elc.copy()
df = df.reset_index()
df['days'] = df.timestamp.apply(lambda x:1 if x.dayofweek > 5 else 0)
df.days.value_counts()

In [ ]:
cal = calendar()
holidays = cal.holidays(start = df.timestamp.min(), end = df.timestamp.max())
df["holiday"] = df.timestamp.isin(holidays).astype('int')
df.head(3)

### X'Mas Week

In [ ]:
xmas = (df.timestamp >= pd.to_datetime("12/20/2016")) & (df.timestamp <= pd.to_datetime('12/27/2016'))


In [ ]:
# # model Architecture
# model = Sequential()
# model.add(LSTM(units=100,
#               return_sequences=True,
#               input_shape=(None, num_x_signals,)))

# model.add(Dense(num_y_signals, activation='sigmoid'))

# model.summary()

#### -- start

As a first step, I instantiate the Sequential class. This will be my model class and I will add LSTM, Dropout and Dense layers to this model. 

In [ ]:
# model Architecture
model_lstm = Sequential()
model_lstm.add(LSTM(units=50,
              return_sequences=True,
#               input_shape=(None, num_x_signals,)
                 input_shape=(None, num_x_signals,)
                ))

model_lstm.add(Dense(num_y_signals, activation='sigmoid'))

# model_s.summary()

the output of the last layer is a tensor with an aritrary batch size and arbitrary sequance length and two output length signals. 

In [ ]:
# from the other code
model_lstm = Sequential()
model_lstm.add(LSTM(units=100,
               return_sequences=True,
               input_shape=(x_batch.shape[1], x_batch.shape[2])))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(num_y_signals))
model_lstm.compile(loss='mean_squared_error', optimizer='adam')

history = model.fit(x_batch, y_batch, epochs=2, 
                    batch_size=70, validation_data=(X_test, Y_test), 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=1, shuffle=False)

# # Training Phase

model.fit_generator(generator=generator,
                    epochs=2,
                    steps_per_epoch=100,
                    validation_data=validation_data,
                    callbacks=callbacks)

In [ ]:
# model = Sequential()
# model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(Dropout(0.2))
# model.add(Dense(1))
# model.compile(loss='mean_squared_error', optimizer='adam')

# history = model.fit(X_train, Y_train, epochs=20, batch_size=70, validation_data=(X_test, Y_test), 
#                     callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=1, shuffle=False)

# # Training Phase
# model.summary()

#### - end

In [ ]:
# def generator(features, labels, batch_size):
    
#     # Create empty arrays to contain batch of features and labels#
#     batch_features = np.zeros((batch_size, 64, 64, 3))
#     batch_labels = np.zeros((batch_size,1))
#     while True:
#         for i in range(batch_size):
#             # choose random index in features
#             index= random.choice(len(features),1)
#             batch_features[i] = some_processing(features[index])
#             batch_labels[i] = labels[index]
#         yield batch_features, batch_labels

# Using series_to_supervised function:

In [ ]:
import supervised_learning as ls

In [ ]:
values = df_elc.values
print(values.shape)
print(type(df_elc.values))

In [ ]:
# ensuring all the data is a float 
values = values.astype('float32')

# normalize features
print('Min and Max values data')
print('    Min:', np.min(values))
print('    Max:', np.max(values))
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
print('Min and Max scaled data')
print('    Min:', np.min(scaled))
print('    Max:', np.max(scaled))

In [ ]:
# frame as supervised learning
reframed = ls.series_to_supervised(scaled, 0, 2)
reframed.head()


In [ ]:
# drop columns we don't want to predict
reframed.drop(reframed.columns[[9,10,11,12,13,14,15]], axis=1, inplace=True)
print(reframed.head())

# LSTM

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [ ]:
from numpy import array

In [ ]:
df = df_elc.iloc[:,0:3].head(20)
dataset = df.values
X_train, y_train = split_sequences(dataset, 3, 1 )

In [ ]:
# X_train

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
# from keras.layers import Dropout
model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(Dropout(0.2))
model.add(Dense(2))
model.compile(loss='mean_squared_error', optimizer='adam')

history = model.fit(X_train, y_train, epochs=20, batch_size=4, #validation_data=(X_test, Y_test), 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=1, shuffle=False)

# Training Phase
model.summary()

In [ ]:
# summarize the data
for i in range(len(X_train)):
    print(X_train[i], y_train[i])

In [ ]:
# from the other code
model_lstm = Sequential()
model_lstm.add(LSTM(units=100,
               return_sequences=True,
               input_shape=(X_train.shape[1], X_train.shape[2])))
# model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(X_train.shape[2]))
model_lstm.compile(loss='mean_squared_error', optimizer='adam')

history = model.fit(X_train, y_train, epochs=2, 
                    batch_size=4, validation_data=(X_test, Y_test), 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=1, shuffle=False)


In [ ]:
# configure network
n_batch = len(X_train)
n_epoch = 100
n_neurons = 10
# design network
model = Sequential()
model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X_train.shape[1], 
                                             X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
df.iloc[:,0:5]